# 데이터 전처리 함수 정리
### 1. load_data : 데이터를 로드하고 결측치를 제거하는 함수
- input : None
- output : 전체 데이터셋, 질문 리스트, 답변 리스트

### 2. clean_text: 질문과 답변 리스트에 대해 통합 전처리를 진행하는 함수
- input : list (question or answer)
- output : filtered list

### 3. filter_normal : 답변 데이터셋 을 전문의 답변과 일반인 답변으로 나눠주는 함수
- 전문의 답변과 그에 대한 인덱스 값, 일반인 답변과 그에 대한 인덱스 값을 반환함.
- input : answer list
- output : psy_answer, psy_index, normal_answer, normal_index 

### 4. clean_psy_answer : 전문의 답변 맞춤형 전처리 함수
- input : psy answer list
- output : filtered psy answer list

### 5. custom_clean_answr : 전문의 답변에 대해 커스텀 필터링 (추후 진행예정)
- input : psy answer list
- output : filtered pasy answer list

### 6. save_data : 데이터를 dataframe 형태로 만든 후, csv로 저장하는 함수
- input : questions, annswers, index, filename
- output : dataframe






# 0. Import Pakages

In [ ]:
cd /content/drive/MyDrive/TobigSlang

/content/drive/.shortcut-targets-by-id/122/TobigSlang


In [ ]:
cd '2. Modeling '/'정진모'/'미경'

/content/drive/.shortcut-targets-by-id/122/TobigSlang/2. Modeling /정진모/미경


In [ ]:
ls

data/  models/  practice/


In [ ]:
import numpy as np
import pandas as pd
import re
import math

# 1. 데이터 로드 및 결측치 제거

In [ ]:
#load_data : 데이터를 불러오고, 결측값을 제거합니다.
def load_data():
  kin = pd.read_csv("data/kin_1130_utf8sig.csv", encoding='utf-8')
  print("Data load success! \nTotal row is", len(kin))

  #remove missing value 
  kin = kin.dropna(axis=0).reset_index(drop=True)
  print("\nAfter removing missing values, the total number of rows is ", len(kin))

  #split data
  questions = kin['question']
  answers = kin['answer']

  return kin, questions, answers

In [ ]:
kin, questions, answers = load_data()

Data load success! 
Total row is 32760

After removing missing values, the total number of rows is  32759


# 2. 특수문자 제거

In [ ]:
def clean_text(texts):
  removed_list = []
  for i in range(0, len(texts)): 
    text = re.sub(r'[@%\\*=()/~&\+á?\xc3\xa1\-\|\:\;\!\-\,\_\~\$\'\"\“]', '',str(texts[i])) #remove punctuation 
    text = re.sub(r'\d+','', text)# remove number
    text = re.sub(r'\.\.+', '.', text) #점이 두 개 이상 반복되는 부분 제거
    text = re.sub(r'\u200B', '', text) #폭 없는 공백 제거
    text = text.lower() #lower case 
    text = re.sub(r'\s+', ' ', text) #remove extra space 
    text = re.sub(r'<[^>]+>','',text) #remove Html tags
    text = re.sub(r'\s+', ' ', text) #remove spaces
    text = re.sub(r"^\s+", '', text) #remove space from start 
    text = re.sub(r'\s+$', '', text) #remove space from the end
    removed_list.append(text)

  return removed_list

In [ ]:
c_questions = clean_text(questions)
c_answers = clean_text(answers)

# 3. Answr Cleansing
이 함수에서는 다음과 같은 순서로 답변을 전처리합니다.
1. 전문인 외 답변 제거
2. 인삿말, 해시태그와 출처링크 삭제
4. 기타 문구 제거


## 3-1. 전문인 외 답변 제거

In [ ]:
# 하나의 단어에 대해서만 필터링합니다.
def filter_normal(answers):
  filter_word = ['신홍범','권순모','배성범','최성환','김봉수','김슬기','한경호','황선희','최인광','이승훈',
                  '박헌구','박태영','박정수','유은정','장현채','신재현','최원석','이춘수','권용석','성기수',
                  '이재원','이상수','김만희','김재옥','신상헌','이형곤','윤충한','박준혁','최영훈','손홍석',
                  '김윤석','김상은','정성호','이희창','임대창','임찬영','조연수','박준헌','김세웅','한명훈',
                  '정우열','김동현','이우철','성종호','조현식','이상경','송미선','강성민','이정석','서경란',
                  '김은지','이종헌','임준석','이경수','서효석','김선아','조우동','백용수','김수룡','박준현',
                  '유지희','안경진','안현웅','고은상','박종석','최정식','오윤정','한혜성','김성재','최순호',
                  '김강률','김남준','김현철','이현제','박병선','이지혜','이희상','장혜련','최상헌','윤혜연',
                  '한병득','이동한','박지웅','정정엽']
  psy = []
  psy_index = []
  normal = []
  normal_index = []
  for i in range(len(answers)):
    if any(w in answers[i] for w in filter_word):
      psy.append(answers[i])
      psy_index.append(i)
    else:
      normal.append(answers[i])
      normal_index.append(i)

  print("Filtering is finished.")
  print("- 총 답변 개수 : ", len(answers))
  print("- 전문의 답변 개수 : ", len(psy))
  print("- 일반인 답변 개수 : ", len(normal))
  print("- 개수 합 일치 여부 : ", len(psy)+len(normal) == len(c_answers))
  return psy, psy_index, normal, normal_index

In [ ]:
# index는 나중에 question과 매칭시킬 때 사용한다.
psy_answer, psy_index, normal_answer, normal_index = filter_normal(c_answers)

Filtering is finished.
- 총 답변 개수 :  32759
- 전문의 답변 개수 :  13562
- 일반인 답변 개수 :  19197
- 개수 합 일치 여부 :  True


In [ ]:
psy_answer[11729]

'안녕하세요. 정신건강연구소 정신건강의학과 전문의 김재옥 입니다.상황에 따라 다르겠지만 불필요해 보입니다.'

## 3-2. 인삿말, 해시태그, 출처 등 삭제

In [ ]:
def clean_psy_answer(texts):
  #1. remove greeting
  pattern1 = '^안녕하세요.{,50}입니다.'
  pattern2 = '^안녕하십니까.{,50}입니다.'

  remove_greeting1 = []
  for i in range(0, len(texts)): 
    text = re.sub(pattern1, '', texts[i])
    remove_greeting1.append(text)
    
  remove_greeting2 = []
  for i in range(0, len(remove_greeting1)): 
    text = re.sub(pattern2, '', remove_greeting1[i])
    remove_greeting2.append(text)

  print("removing greeting completed.")

  #2. 위에서 공통적으로 보이는 패턴을 수정합니다.
      # a. 앞과 뒤의 white space
      # b. .뒤에 space가 없는 애들은 space 넣어줄 것.
      # c. .앞 뒤로 글자가 없는 애들은 지워줄 것.
  
  remove_pattern = []
  for i in range(0, len(remove_greeting2)): 
    text = re.sub(r'^\.', '',remove_greeting2[i]) #맨 앞에 문자가 아닌 것이 등장한 경우
    text = re.sub(r'\s+\.$', '', text) #맨 뒤에 문자가 아닌 것이 등장한 경우 
    text = re.sub(r'\s+\.\s+', '', text) #무의미한 .이 들어있는 경우 - (3)번
    text = re.sub(r"^\s+", '', text) #remove space from start 
    text = re.sub(r'\s+$', '', text) #remove space from the end

    # (2)번 마침표 뒤에 스페이스 없는 경우 스페이스 추가.
    while re.search('[가-힣]\.\S', text):
      mats = re.search('\w\.\S', text)
      start = mats.span()[0]
      end = mats.span()[1]
      text = text[:start+2]+' '+text[end-1:]
    remove_pattern.append(text)
  print("removing repeated pattern completed.")

  #3. 해시태그, 출처, url 제거
  removed_list = []
  for i in range(0, len(remove_pattern)): 
    text = re.sub(r'#\w+', '',remove_pattern[i]) #해시태그 삭제
    text = re.sub(r'출처', '', text) #출처 단어 삭제
    text = re.sub(r"[a-z]+\.", '', text) #이메일 삭제
    text = re.sub(r"\[.+\]", '', text) #[문자] 형태 삭제
    text = re.sub(r"\s+", ' ', text) #위의 단어들이 삭제되면서 생긴 white space 삭제.
    text = re.sub(r"^\s+", '', text) #remove space from start 
    text = re.sub(r'\s+$', '', text) #remove space from the end
    removed_list.append(text)
    
  print("removing special pattern completed.")

  print("all cleaning answer process is finished.")
  print("length of data is", len(removed_list))
  return removed_list

In [ ]:
psy_answer = clean_psy_answer(psy_answer)

removing greeting completed.
removing repeated pattern completed.
removing special pattern completed.
all cleaning answer process is finished.
length of data is 13562


## 3-3. 기타 문구 제거
육안으로 확인하면서 반복되는 패턴 등을 추가하여 삭제할 예정

In [ ]:
def custom_clean_answer(texts):
  pass

# 4. 데이터 저장
완성된 answer와 그에 해당되는 question을 저장합니다.

In [ ]:
cd /content/drive/MyDrive/TobigSlang/'1. Data'/'정진모'/split_answer

/content/drive/.shortcut-targets-by-id/122/TobigSlang/1. Data/정진모/split_answer


In [ ]:
def save_data(questions, answer, index, filename):
  
  if filename == "normal":
    print("\nIs this normal answer?")
    normal = []
    for i in range(len(answer)):
      normal.append(answer[i])
    answer = normal

  #answer의 종류에 맞는 question만 골라내기
  question = []
  for i in range(len(questions)):
    if i in index:
      question.append(questions[i])

  #to dataframe
  dic = {'question': question, 'answer': answer}
  df = pd.DataFrame(data=dic)

  #to csv
  df.to_csv(filename+".csv", index=False)

  print("Saving", filename, "data is finished.")
  print("- 질문 개수 : ", len(question))
  print("- 답변 개수 : ", len(answer))

  return df

In [ ]:
psy = save_data(c_questions, psy_answer, psy_index, "psy")
normal = save_data(c_questions, normal_answer, normal_index, "normal")

Saving psy data is finished.
- 질문 개수 :  13562
- 답변 개수 :  13562

Is this normal answer?
Saving normal data is finished.
- 질문 개수 :  19197
- 답변 개수 :  19197


# 5. 데이터 불러오기 

In [ ]:
temp = pd.read_csv("psy.csv", encoding="utf-8")
temp

,question,answer
0,안녕하세요 대 초반 여자입니다남자친구랑 같이 살고있는데요 남자친구가 항상 제가 잠꼬...,잠꼬대를 하시는 분들이 있고요. 잠꼬대가 심한 경우에 옆 사람이 놀라죠. 본인은 잘...
1,연세가 이신 할머니께서 한 년 전 부터 자도 자도 피곤하다고 하셔요. 안 잔거 같고...,잠꼬대를 심하게 하는 사람은 치매에 걸릴 위험이 높다. 사실일까요 실제로 이를 뒷받...
2,약 일전부터. 악몽을계속꿉니다. 낮이든 밤이든 잠에들면 꾸고 저는항상 잠귀가밝아 항...,가위눌림을 경험하신 분들이 있으시죠. 잠을 자다가 갑자기 머리는 깼는데 몸은 아직 ...
3,제가 우울증에 불안증까지 있어서 정신병원다니면서 약을 먹고있는 상태입니다. 근데 며...,가위눌림을 경험하신 분들이 있으시죠. 잠을 자다가 갑자기 머리는 깼는데 몸은 아직 ...
4,제가 잠을 시간 정도는 자는데 고등학생 치고 그렇게 적게자는건 아닌거 같거든요. 근...,을 너무 많이 잔다 도대체 내가 무슨 문제가 있나 이렇게 질문하고 걱정하시는 분들이...
...,...,...
13557,제가 아주 어렷을적부터 혼자 대화하는듯이 울고 웃거나 같이살던 할머니를 자주 때렷고...,말씀하셨던 증상의 시기 및 증상의 양상을 볼 때 조현병에 해당하는 것으로 사료됩니다...
13558,주일 근무하구요 일하는 시간에는 사람들과 같이 밥먹는게불편합니다 주로 먹는게 마시는...,상황에 따른 식욕 변화에 대해서 질문하시는 거군요. 일단 음식을 불규칙하게 섭취하는...
13559,시에자면 시에일어났다가 다시자고 시에 일어났다가 다시자면 시에 일어나요.왜이러는지 ...,수면 유지가 잘 되지 않아서 힘드시군요. 수면유지를 방해하는 경우는 여러가지 이유가...
13560,살 남자입니다.일단 저는 제가 우울증이라 생각한적이 없습니다. 커오면서 외로움도 느...,마치 점점 추워지는 겨울처럼 알게 모르게 질문자님의 마음은 더 메마르고 힘겨워지고 ...


# (수정) 12/27 결측치 추가 제거
question : 7971  
answer : 11729  


In [ ]:
temp2 = temp.dropna(axis=0).reset_index(drop=True)
temp2

,question,answer
0,안녕하세요 대 초반 여자입니다남자친구랑 같이 살고있는데요 남자친구가 항상 제가 잠꼬...,잠꼬대를 하시는 분들이 있고요. 잠꼬대가 심한 경우에 옆 사람이 놀라죠. 본인은 잘...
1,연세가 이신 할머니께서 한 년 전 부터 자도 자도 피곤하다고 하셔요. 안 잔거 같고...,잠꼬대를 심하게 하는 사람은 치매에 걸릴 위험이 높다. 사실일까요 실제로 이를 뒷받...
2,약 일전부터. 악몽을계속꿉니다. 낮이든 밤이든 잠에들면 꾸고 저는항상 잠귀가밝아 항...,가위눌림을 경험하신 분들이 있으시죠. 잠을 자다가 갑자기 머리는 깼는데 몸은 아직 ...
3,제가 우울증에 불안증까지 있어서 정신병원다니면서 약을 먹고있는 상태입니다. 근데 며...,가위눌림을 경험하신 분들이 있으시죠. 잠을 자다가 갑자기 머리는 깼는데 몸은 아직 ...
4,제가 잠을 시간 정도는 자는데 고등학생 치고 그렇게 적게자는건 아닌거 같거든요. 근...,을 너무 많이 잔다 도대체 내가 무슨 문제가 있나 이렇게 질문하고 걱정하시는 분들이...
...,...,...
13555,제가 아주 어렷을적부터 혼자 대화하는듯이 울고 웃거나 같이살던 할머니를 자주 때렷고...,말씀하셨던 증상의 시기 및 증상의 양상을 볼 때 조현병에 해당하는 것으로 사료됩니다...
13556,주일 근무하구요 일하는 시간에는 사람들과 같이 밥먹는게불편합니다 주로 먹는게 마시는...,상황에 따른 식욕 변화에 대해서 질문하시는 거군요. 일단 음식을 불규칙하게 섭취하는...
13557,시에자면 시에일어났다가 다시자고 시에 일어났다가 다시자면 시에 일어나요.왜이러는지 ...,수면 유지가 잘 되지 않아서 힘드시군요. 수면유지를 방해하는 경우는 여러가지 이유가...
13558,살 남자입니다.일단 저는 제가 우울증이라 생각한적이 없습니다. 커오면서 외로움도 느...,마치 점점 추워지는 겨울처럼 알게 모르게 질문자님의 마음은 더 메마르고 힘겨워지고 ...
